# REGISTRAZIONE MNI152 IXI VERSION

In [ ]:
import os
from pathlib import Path
import glob
import ants
import sys

# proviamo a importare tqdm per la progress bar
try:
    from tqdm import tqdm
except ImportError:
    tqdm = None

# ======================================================
# CONFIG
# ======================================================
SRC_ROOT = r"C:\Users\Stefano\Desktop\Stefano\Datasets\IXI-SKULLSTRIPPED\filtered_subjects\skullstripped_subjects_coregistered_affine"
DST_ROOT = r"C:\Users\Stefano\Desktop\Stefano\Datasets\IXI-SKULLSTRIPPED\IXI_SKULLSTRIPPED_MNI"

MNI_T1 = r"C:\Users\Stefano\Desktop\Stefano\Codice_skull\Auto-SkullStripping\Mni152Registration\mni152\icbm_avg_152_t1_tal_nlin_symmetric_VI_mask.nii"

TYPE_M2T1 = "Rigid"   # "Rigid" | "AffineFast" | "Affine"

TFM_ROOT = Path(DST_ROOT) / "_Transforms"
TFM_ROOT.mkdir(parents=True, exist_ok=True)


# ======================================================
# 1. Trova i soggetti
# ======================================================
def discover_subject_folders(root: str):
    root_p = Path(root)
    return [p.name for p in sorted(root_p.iterdir(), key=lambda x: x.name) if p.is_dir()]


# ======================================================
# 2. Helpers
# ======================================================
def ensure_parent_dir(p: Path):
    p.parent.mkdir(parents=True, exist_ok=True)


def normalized_mni_name(subject: str, modality: str) -> str:
    return f"{subject}_{modality.upper()}_mni.nii.gz"


def clean_subject_name(src_name: str) -> str:
    """
    Se la cartella sorgente si chiama 'skullstripped_IXI002-Guys-0828'
    qui la trasformiamo in 'IXI002-Guys-0828'
    """
    if src_name.lower().startswith("skullstripped_"):
        return src_name[len("skullstripped_"):]
    return src_name


def find_modality_file(folder: Path, modality: str) -> Path | None:
    """
    Regola:
    - T1  -> contiene 't1' ma NON 't2' e NON 'pd'
    - T2  -> contiene 't2'
    - PD  -> contiene 'pd'
    escludo solo le vere maschere '*_mask.nii', NON i file '*_masked.nii'
    """
    modality = modality.upper()
    files = list(folder.glob("*.nii")) + list(folder.glob("*.nii.gz"))

    for f in files:
        name = f.name.lower()

        # escludo solo maschere pure
        if "mask" in name and "masked" not in name:
            continue

        if modality == "T1":
            if "t1" in name and "t2" not in name and "pd" not in name:
                return f

        elif modality == "T2":
            if "t2" in name:
                return f

        elif modality == "PD":
            if "pd" in name:
                return f

    return None


# ======================================================
# 2b. Controllo e risampling a 1 mm3
# ======================================================
def ensure_iso_1mm_image(img: ants.ANTsImage):
    spacing = img.spacing
    if all(abs(s - 1.0) < 1e-3 for s in spacing):
        return img, False
    img_res = ants.resample_image(img, (1, 1, 1), use_voxels=False, interp_type=1)  # NN
    return img_res, True


# ======================================================
# 3. trasformazioni
# ======================================================
def t1_outprefix_for_transforms(subject: str, t1_name: str) -> Path:
    base = t1_name
    if base.endswith(".nii.gz"):
        base = base[:-7]
    elif base.endswith(".nii"):
        base = base[:-4]
    return TFM_ROOT / f"{subject}_{base}_T1toMNI_"


def find_saved_t1_to_mni_transforms(prefix: Path) -> list[str] | None:
    warp_candidates = glob.glob(str(prefix) + "*Warp.nii*")
    aff_candidates = glob.glob(str(prefix) + "*GenericAffine.mat")
    warp = sorted(warp_candidates, key=len)[:1]
    aff = sorted(aff_candidates, key=len)[:1]
    if warp and aff:
        return [warp[0], aff[0]]
    return None


def register_t1_to_mni_full(mni_img: ants.ANTsImage, t1_img: ants.ANTsImage, outprefix: Path | None = None):
    reg_kwargs = dict(fixed=mni_img, moving=t1_img, type_of_transform="SyN")
    if outprefix is not None:
        ensure_parent_dir(outprefix.with_suffix(".sentinel"))
        reg_kwargs["outprefix"] = str(outprefix)
    reg = ants.registration(**reg_kwargs)
    fwd = reg["fwdtransforms"]
    warp = next((p for p in fwd if p.endswith("Warp.nii.gz") or p.endswith("Warp.nii")), None)
    aff = next((p for p in fwd if p.endswith("GenericAffine.mat")), None)
    if aff is None or warp is None:
        raise RuntimeError("SyN non ha prodotto sia warp che affine.")
    return reg["warpedmovout"], [warp, aff]


def register_mod_to_t1_affine(mod_img: ants.ANTsImage, t1_img: ants.ANTsImage, mode: str):
    reg = ants.registration(fixed=t1_img, moving=mod_img, type_of_transform=mode)
    mat = reg["fwdtransforms"][-1]
    return mat


def push_to_mni(moving_img, mni_img, tf_chain, is_label=False):
    return ants.apply_transforms(
        fixed=mni_img,
        moving=moving_img,
        transformlist=tf_chain,
        interpolator=("nearestNeighbor" if is_label else "linear"),
    )


# ======================================================
# 4. MAIN
# ======================================================
def main():
    src_root = Path(SRC_ROOT)
    dst_root = Path(DST_ROOT)
    dst_root.mkdir(parents=True, exist_ok=True)

    mni_img = ants.image_read(MNI_T1)

    subjects = discover_subject_folders(SRC_ROOT)
    total = len(subjects)
    print(f"[INFO] Soggetti trovati: {total}")

    it = subjects
    if tqdm is not None:
        it = tqdm(subjects, desc="Registrazione su MNI", unit="soggetto")

    for idx, subject in enumerate(it, start=1):
        if tqdm is None:
            print(f"\n[{idx}/{total}] === {subject} ===")
        else:
            print(f"\n=== {subject} ===")

        src_subj_dir = src_root / subject

        # pulizia del nome soggetto in output
        clean_name = clean_subject_name(subject)

        # nuova struttura di destinazione:
        # DST_ROOT / sub_<clean_name> / anat / skullstripped
        dst_subj_dir = Path(DST_ROOT) / f"sub_{clean_name}" / "anat" / "skullstripped"
        dst_subj_dir.mkdir(parents=True, exist_ok=True)

        # -------- T1 --------
        t1_path = find_modality_file(src_subj_dir, "T1")
        if t1_path is None:
            print("  ⚠️ Nessuna T1 'pulita' trovata (solo T1), salto questo soggetto.")
            continue
        print(f"  ✅ T1 trovata: {t1_path.name}")

        # carico T1 e porto a 1mm
        t1_img = ants.image_read(str(t1_path))
        t1_img, was_resampled = ensure_iso_1mm_image(t1_img)
        if was_resampled:
            print("  🔁 T1 risamplata a 1mm³ (nearest).")

        # prefisso per le trasformazioni (uso il nome originale del soggetto per coerenza)
        tfm_prefix = t1_outprefix_for_transforms(subject, t1_path.name)

        # nome di output normalizzato (uso il nome pulito!)
        t1_outname = normalized_mni_name(clean_name, "T1")
        t1_dst = dst_subj_dir / t1_outname

        # T1 → MNI
        t1_to_mni_chain = None

        if t1_dst.exists():
            print(f"  ⏩ T1 già registrata: {t1_dst.name}")
            saved = find_saved_t1_to_mni_transforms(tfm_prefix)
            if saved:
                print(f"  🔄 Ri-uso trasformazioni già salvate: {saved}")
                t1_to_mni_chain = saved
            else:
                print("  ⚠️ Trasformazioni non trovate, le rigenero...")
                t1_in_mni, t1_to_mni_chain = register_t1_to_mni_full(mni_img, t1_img, outprefix=tfm_prefix)
        else:
            print("  ▶ Registro T1 → MNI (SyN)...")
            t1_in_mni, t1_to_mni_chain = register_t1_to_mni_full(mni_img, t1_img, outprefix=tfm_prefix)
            ants.image_write(t1_in_mni, str(t1_dst))
            print(f"  ✅ Salvata T1 in MNI: {t1_dst.name}")

        if not t1_to_mni_chain:
            print("  ❌ Non ho una catena T1→MNI valida, salto le altre modalità.")
            continue

        # -------- T2 e PD --------
        for mod in ("T2", "PD"):
            mod_path = find_modality_file(src_subj_dir, mod)
            if mod_path is None:
                print(f"  ℹ️ {mod} non trovata, salto.")
                continue

            mod_outname = normalized_mni_name(clean_name, mod)
            mod_dst = dst_subj_dir / mod_outname

            if mod_dst.exists():
                print(f"  ⏩ {mod} già registrata: {mod_dst.name}")
                continue

            print(f"  ▶ Registro {mod}: {mod_path.name}")

            mod_img = ants.image_read(str(mod_path))
            mod_img, was_resampled_mod = ensure_iso_1mm_image(mod_img)
            if was_resampled_mod:
                print(f"    🔁 {mod} risamplata a 1mm³ (nearest).")

            # mod → T1
            mod2t1_aff = register_mod_to_t1_affine(mod_img, t1_img, TYPE_M2T1)

            # catena completa
            full_chain = [*t1_to_mni_chain, mod2t1_aff]

            mod_in_mni = push_to_mni(mod_img, mni_img, full_chain, is_label=False)
            ants.image_write(mod_in_mni, str(mod_dst))
            print(f"    ✅ Salvata {mod} in MNI: {mod_dst.name}")

    print("\n[TUTTO FATTO] Registrazione su MNI completata.")
    return 0


if __name__ == "__main__":
    sys.exit(main())


[INFO] Soggetti trovati: 577


Registrazione su MNI:   0%|          | 1/577 [00:00<01:03,  9.01soggetto/s]


=== skullstripped_IXI002-Guys-0828 ===
  ✅ T1 trovata: IXI002-Guys-0828_T1_iso05-stripped.nii.gz
  ⏩ T1 già registrata: IXI002-Guys-0828_T1_mni.nii.gz
  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI002-Guys-0828_IXI002-Guys-0828_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI002-Guys-0828_IXI002-Guys-0828_T1_iso05-stripped_T1toMNI_0GenericAffine.mat']
  ⏩ T2 già registrata: IXI002-Guys-0828_T2_mni.nii.gz
  ⏩ PD già registrata: IXI002-Guys-0828_PD_mni.nii.gz

=== skullstripped_IXI012-HH-1211 ===
  ✅ T1 trovata: IXI012-HH-1211_T1_iso05-stripped.nii.gz
  ⏩ T1 già registrata: IXI012-HH-1211_T1_mni.nii.gz

Registrazione su MNI:   1%|          | 3/577 [00:00<01:05,  8.70soggetto/s]


  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI012-HH-1211_IXI012-HH-1211_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI012-HH-1211_IXI012-HH-1211_T1_iso05-stripped_T1toMNI_0GenericAffine.mat']
  ⏩ T2 già registrata: IXI012-HH-1211_T2_mni.nii.gz
  ⏩ PD già registrata: IXI012-HH-1211_PD_mni.nii.gz

=== skullstripped_IXI013-HH-1212 ===
  ✅ T1 trovata: IXI013-HH-1212_T1_iso05-stripped.nii.gz
  ⏩ T1 già registrata: IXI013-HH-1212_T1_mni.nii.gz
  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI013-HH-1212_IXI013-HH-1212_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Tran

...

  ⏩ T1 già registrata: IXI254-HH-1705_T1_mni.nii.gz
  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI254-HH-1705_IXI254-HH-1705_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI254-HH-1705_IXI254-HH-1705_T1_iso05-stripped_T1toMNI_0GenericAffine.mat']
  ⏩ T2 già registrata: IXI254-HH-1705_T2_mni.nii.gz
  ⏩ PD già registrata: IXI254-HH-1705_PD_mni.nii.gz

=== skullstripped_IXI255-HH-1882 ===
  ✅ T1 trovata: IXI255-HH-1882_T1_iso05-stripped.nii.gz
  ⏩ T1 già registrata: IXI255-HH-1882_T1_mni.nii.gz
  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI255-HH-1882_IXI255-HH-1882_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datase

Registrazione su MNI:  39%|███▉      | 224/577 [00:27<00:43,  8.03soggetto/s]

  ⏩ T1 già registrata: IXI256-HH-1723_T1_mni.nii.gz
  🔄 Ri-uso trasformazioni già salvate: ['C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI256-HH-1723_IXI256-HH-1723_T1_iso05-stripped_T1toMNI_1Warp.nii.gz', 'C:\\Users\\Stefano\\Desktop\\Stefano\\Datasets\\IXI-SKULLSTRIPPED\\IXI_SKULLSTRIPPED_MNI\\_Transforms\\skullstripped_IXI256-HH-1723_IXI256-HH-1723_T1_iso05-stripped_T1toMNI_0GenericAffine.mat']
  ⏩ T2 già registrata: IXI256-HH-1723_T2_mni.nii.gz
  ⏩ PD già registrata: IXI256-HH-1723_PD_mni.nii.gz

=== skullstripped_IXI257-HH-1724 ===
  ✅ T1 trovata: IXI257-HH-1724_T1_iso05-stripped.nii.gz
  ▶ Registro T1 → MNI (SyN)...
  ✅ Salvata T1 in MNI: IXI257-HH-1724_T1_mni.nii.gz
  ▶ Registro T2: IXI257-HH-1724_T2_coreg_on_T1iso05_masked.nii.gz
    ✅ Salvata T2 in MNI: IXI257-HH-1724_T2_mni.nii.gz
  ▶ Registro PD: IXI257-HH-1724_PD_coreg_on_T1iso05_masked.nii.gz


Registrazione su MNI:  39%|███▉      | 225/577 [01:11<1:19:06, 13.48s/soggetto]

    ✅ Salvata PD in MNI: IXI257-HH-1724_PD_mni.nii.gz

=== skullstripped_IXI258-HH-1769 ===
  ✅ T1 trovata: IXI258-HH-1769_T1_iso05-stripped.nii.gz
  ▶ Registro T1 → MNI (SyN)...


# EXTRACT AXIAL DA PROVARE, CON QUESTO POSSO IMPOSTARE QUANTI SOGGETTI DA PRENDERE COME ADDESTRAMENTO

In [ ]:
import os
import re
import shutil
import nibabel as nib
import numpy as np
from skimage.transform import resize
from nibabel.processing import resample_to_output
import imageio
from pathlib import Path
import random
from datetime import datetime

# proviamo a usare tqdm per la barra di avanzamento
try:
    from tqdm import tqdm
except ImportError:
    tqdm = None

# ========================================
# CONFIG
# ========================================
base_root = r"C:\Users\Stefano\Desktop\Stefano\Datasets\IXI-SKULLSTRIPPED\IXI_SKULLSTRIPPED_MNI"

# 🔹 opzioni di suddivisione
N_SUBJECTS_FOR_TRAIN = 280   # se None → usa tutto il dataset e fa split 80/20
MAKE_TRAIN_FOLDERS = True    # se False → non crea/scrive trainA e trainB

# 🔹 output globale per il TRAIN
TRAIN_ROOT = r"C:\Users\Stefano\Desktop\Stefano\Datasets\IXI-SKULLSTRIPPED\TRAINING_IMAGES_T1_PD_IXI\subjects\train"
GLOBAL_TRAIN_A = os.path.join(TRAIN_ROOT, "trainA")  # T1
GLOBAL_TRAIN_B = os.path.join(TRAIN_ROOT, "trainB")  # T2
TARGET_MODALITY_trainA = "T1"
TARGET_MODALITY_trainB = "PD"

# 🔹 output per il TEST
TEST_ROOT = r"C:\Users\Stefano\Desktop\Stefano\Datasets\IXI-SKULLSTRIPPED\TRAINING_IMAGES_T1_PD_IXI\subjects\train"

if MAKE_TRAIN_FOLDERS:
    os.makedirs(GLOBAL_TRAIN_A, exist_ok=True)
    os.makedirs(GLOBAL_TRAIN_B, exist_ok=True)
os.makedirs(TEST_ROOT, exist_ok=True)

# 🔴 se False non sovrascrive i PNG esistenti
OVERWRITE = True

# modalità di riferimento per il crop
ref_modality = "T1"

# parametri
nz_threshold = 5000
resize_shape = (256, 256)
voxel_sizes = (1, 1, 1)

random.seed(42)

# ========================================
# DISCOVERY DELLE CARTELLE
# ========================================
def discover_subject_anat_pairs(root: str):
    """
    Ritorna tutte le coppie (subj, path_relativo) dove esiste anat/skullstripped.
    """
    root_p = Path(root)
    for subj in sorted(root_p.iterdir(), key=lambda p: p.name):
        if not subj.is_dir():
            continue
        if not subj.name.lower().startswith("sub"):
            continue

        anat_dir = subj / "anat"
        if anat_dir.is_dir():
            skull = anat_dir / "skullstripped"
            if skull.is_dir():
                yield (subj.name, "anat/skullstripped")


# ========================================
# PATH PER LE MODALITÀ (dentro skullstripped)
# ========================================
def path_modality(subject: str, rel_patient: str, modality: str):
    folder = os.path.join(base_root, subject, rel_patient)
    if not os.path.isdir(folder):
        return None

    modality = modality.upper()
    for f in os.listdir(folder):
        f_low = f.lower()
        if not (f_low.endswith(".nii") or f_low.endswith(".nii.gz")):
            continue
        if "mask" in f_low:
            continue
        if f_low.endswith(f"_{modality.lower()}_mni.nii.gz") or f_low.endswith(f"_{modality.lower()}_mni.nii"):
            return os.path.join(folder, f)
    return None


# ========================================
# OUTPUT LOCALE
# ========================================
def output_root(subject: str, rel_patient: str) -> str:
    return os.path.join(base_root, subject, rel_patient, "Output", "ALL_AXES")


# ========================================
# FUNZIONI DI ELABORAZIONE
# ========================================
def load_and_resample(path, is_mask=False):
    img = nib.load(path)
    order = 0 if is_mask else 1
    resampled_img = resample_to_output(img, voxel_sizes=voxel_sizes, order=order)
    data = resampled_img.get_fdata()
    data[data < 0] = 0

    if is_mask:
        return data

    mean = np.mean(data)
    std = np.std(data)
    if std > 0:
        data = (data - mean) / std
    else:
        data = np.zeros_like(data)

    dmin, dmax = np.nanmin(data), np.nanmax(data)
    if dmax > dmin:
        data = (data - dmin) / (dmax - dmin)
    else:
        data = np.zeros_like(data)

    return data


def compute_crop_and_valid_slices(ref_vol, orientation, nz_threshold, resize_shape):
    if orientation == 'axial':
        mask2d = np.any(ref_vol > 0, axis=2)
        slicer = lambda idx: ref_vol[:, :, idx]
        n_slices = ref_vol.shape[2]
    elif orientation == 'coronal':
        mask2d = np.any(ref_vol > 0, axis=1)
        slicer = lambda idx: ref_vol[:, idx, :]
        n_slices = ref_vol.shape[1]
    elif orientation == 'sagittal':
        mask2d = np.any(ref_vol > 0, axis=0)
        slicer = lambda idx: ref_vol[idx, :, :]
        n_slices = ref_vol.shape[0]
    else:
        raise ValueError("orientation deve essere 'axial', 'coronal' o 'sagittal'.")

    coords = np.argwhere(mask2d)
    if coords.size == 0:
        raise ValueError(f"Nessun voxel non-zero nel volume di riferimento per {orientation}")

    ymin, xmin = coords.min(axis=0)
    ymax, xmax = coords.max(axis=0)

    valid_indices = []
    for i in range(n_slices):
        slice_2d = slicer(i)
        cropped = slice_2d[ymin:ymax+1, xmin:xmax+1]
        resized = resize(cropped, resize_shape, preserve_range=True,
                         anti_aliasing=True, order=1)
        out_uint8 = (resized * 255).astype(np.uint8)
        if np.count_nonzero(out_uint8) >= nz_threshold:
            valid_indices.append(i)

    return (ymin, xmin, ymax, xmax), valid_indices


def build_slice_name(subject, rel_patient, modality, slice_idx, ext=".png"):
    modality = modality.upper()
    return f"{subject}_{modality}_slice_{slice_idx:03d}{ext}"


def save_png_respecting_overwrite(img_array_uint8, dst_folder, filename):
    os.makedirs(dst_folder, exist_ok=True)
    out_path = os.path.join(dst_folder, filename)
    if OVERWRITE or not os.path.exists(out_path):
        imageio.imwrite(out_path, img_array_uint8)


def save_slices_for_orientation(subject,
                                rel_patient,
                                modality,
                                vol,
                                orientation,
                                crop_box,
                                valid_indices,
                                local_out_dir,
                                subject_in_test: bool):
    ymin, xmin, ymax, xmax = crop_box

    if orientation == 'axial':
        slicer = lambda idx: vol[:, :, idx]
    elif orientation == 'coronal':
        slicer = lambda idx: vol[:, idx, :]
    else:
        slicer = lambda idx: vol[idx, :, :]

    os.makedirs(local_out_dir, exist_ok=True)

    count = 0
    for i in valid_indices:
        sl = slicer(i)
        cropped = sl[ymin:ymax+1, xmin:xmax+1]
        resized = resize(cropped, resize_shape, preserve_range=True, anti_aliasing=True)
        out_slice = (resized * 255).astype(np.uint8)
        fname = build_slice_name(subject, rel_patient, modality, i)

        # salva locale
        local_path = os.path.join(local_out_dir, fname)
        if OVERWRITE or not os.path.exists(local_path):
            imageio.imwrite(local_path, out_slice)

        # salva globale (solo se train attivo e solo axial)
        if orientation == "axial" and MAKE_TRAIN_FOLDERS:
            if subject_in_test:
                subj_root = os.path.join(TEST_ROOT, subject, "test")
                if modality.upper() == TARGET_MODALITY_trainA:
                    final_dir = os.path.join(subj_root, "testA")
                elif modality.upper() == TARGET_MODALITY_trainB:
                    final_dir = os.path.join(subj_root, "testB")
                else:
                    final_dir = None
            else:
                if modality.upper() == TARGET_MODALITY_trainA:
                    final_dir = GLOBAL_TRAIN_A
                elif modality.upper() == TARGET_MODALITY_trainB:
                    final_dir = GLOBAL_TRAIN_B
                else:
                    final_dir = None
            if final_dir:
                save_png_respecting_overwrite(out_slice, final_dir, fname)

        count += 1
    return count


# ========================================
# PROCESS DI UN SOGGETTO
# ========================================
def process_subject(subject: str, rel_patient: str, global_crop_valid, subject_in_test: bool):
    print(f"\n=== {subject} | {rel_patient} ===")

    skull_dir = os.path.join(base_root, subject, rel_patient)
    if not os.path.isdir(skull_dir):
        print(f"[SKIP] Nessuna cartella skullstripped qui: {skull_dir}")
        return False, global_crop_valid

    out_root = output_root(subject, rel_patient)
    out_parent = os.path.dirname(out_root)
    if OVERWRITE and os.path.isdir(out_parent):
        shutil.rmtree(out_parent)
    os.makedirs(out_root, exist_ok=True)

    modalities_paths = {}
    for m in ["T1", "T2"]:
        p = path_modality(subject, rel_patient, m)
        if p:
            modalities_paths[m] = p

    if not modalities_paths:
        print(f"[SKIP] Nessuna modalità trovata in {skull_dir}")
        return False, global_crop_valid

    if ref_modality in modalities_paths:
        ref_path = modalities_paths[ref_modality]
    else:
        ref_path = list(modalities_paths.values())[0]

    ref_vol = load_and_resample(ref_path)
    orientations = ["axial", "coronal", "sagittal"]

    if global_crop_valid is None:
        global_crop_valid = {}
        for ori in orientations:
            crop_box, valid_idx = compute_crop_and_valid_slices(ref_vol, ori, nz_threshold, resize_shape)
            global_crop_valid[ori] = (crop_box, valid_idx)
            print(f"[{ori}] Kept {len(valid_idx)} slices (definito come riferimento globale)")
    else:
        print("Uso crop globale già definito.")

    for mod, vol_path in modalities_paths.items():
        mod_dir = os.path.join(out_root, mod)
        os.makedirs(mod_dir, exist_ok=True)
        vol = load_and_resample(vol_path)
        total_saved = 0
        for ori in orientations:
            crop_box, valid_idx = global_crop_valid[ori]
            ori_dir = os.path.join(mod_dir, ori)
            os.makedirs(ori_dir, exist_ok=True)
            saved = save_slices_for_orientation(subject, rel_patient, mod, vol, ori,
                                                crop_box, valid_idx, ori_dir, subject_in_test)
            total_saved += saved
            print(f"Saved {saved} {mod} {ori} slices.")
        print(f"=> {mod}: {total_saved} totali.")
    return True, global_crop_valid


# ========================================
# MAIN
# ========================================
if __name__ == "__main__":
    pairs = list(discover_subject_anat_pairs(base_root))
    subjects_with_pairs = sorted({subj for subj, _ in pairs})
    n_total = len(subjects_with_pairs)

    # logica nuova
    if N_SUBJECTS_FOR_TRAIN is None:
        n_test = max(1, int(n_total * 0.20))
        random.shuffle(subjects_with_pairs)
        test_subjects = set(subjects_with_pairs[:n_test])
        train_subjects = set(subjects_with_pairs[n_test:])
    else:
        if N_SUBJECTS_FOR_TRAIN > n_total:
            raise ValueError(f"Hai chiesto {N_SUBJECTS_FOR_TRAIN} soggetti ma nel dataset ce ne sono solo {n_total}.")
        random.shuffle(subjects_with_pairs)
        train_subjects = set(subjects_with_pairs[:N_SUBJECTS_FOR_TRAIN])
        test_subjects = set(subjects_with_pairs[N_SUBJECTS_FOR_TRAIN:])

    print("==================================================")
    print(f"RUN estrazione: {datetime.now().isoformat(sep=' ', timespec='seconds')}")
    print(f"Soggetti totali: {n_total}")
    print(f"In TRAIN: {len(train_subjects)}")
    print(f"In TEST:  {len(test_subjects)}")
    print(f"MAKE_TRAIN_FOLDERS = {MAKE_TRAIN_FOLDERS}")
    print("==================================================")

    global_crop_valid = None
    missing = []

    pair_iter = tqdm(pairs, desc="Estrazione slice", unit="cartella") if tqdm else pairs

    for idx, (subject, rel_patient) in enumerate(pair_iter, 1):
        ok, global_crop_valid = process_subject(subject, rel_patient, global_crop_valid, subject in test_subjects)
        if not ok:
            missing.append(f"{subject}/{rel_patient}")

    print("\n✅ Estrazione completata.")
    if missing:
        print("\n⚠️ Soggetti senza volumi utili:")
        for s in missing:
            print(" -", s)

    if MAKE_TRAIN_FOLDERS:
        def count_png(folder):
            return sum(1 for _ in Path(folder).glob("*.png"))
        count_A = count_png(GLOBAL_TRAIN_A)
        count_B = count_png(GLOBAL_TRAIN_B)
        print("\n📊 Riepilogo finale (solo train):")
        print(f"  - trainA ({TARGET_MODALITY_trainA}) : {count_A:,} immagini")
        print(f"  - trainB ({TARGET_MODALITY_trainB}) : {count_B:,} immagini")
        diff = abs(count_A - count_B)
        if diff == 0:
            print("✅ Le due cartelle hanno lo stesso numero di immagini.")
        else:
            print(f"⚠️ Differenza di {diff} immagini tra trainA e trainB.")
